In [1]:
import os
os.environ['HTTP_PROXY'] = "http://127.0.0.1:10809"
os.environ['HTTPS_PROXY'] = "http://127.0.0.1:10809"

In [2]:
import geemap
import ee
Map=geemap.Map()
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [7]:
# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)
# AWEI(自动水体提取指数): AWEIsh = B2 + 2.5B3 - 1.5(B5+B6) - 0.25B7
def water_index(image):
    ndvi=image.normalizedDifference(['B5', 'B4']).rename('NDVI')
    mndwi=image.normalizedDifference(['B3', 'B6']).rename("mNDWI") 
    ndvi_mndwi = ndvi.subtract(mndwi).rename('ndvi_mndwi')
    awei = image.expression('B2 + 2.5*B3 - 1.5*(B5+B6) - 0.25*B7',
        {
          'B2': image.select('B2'),
          'B3': image.select('B3'),    
          'B5': image.select('B5'),    
          'B6': image.select('B6'),
          'B7': image.select('B7'),
        }).rename('AWEI')
    return image.addBands(ndvi_mndwi).addBands(ndvi).addBands(mndwi).addBands(awei)
visParams = {
    'bands': ['B5', 'B6', 'B4'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

In [13]:
region = ee.Geometry.Rectangle([90.122,34.629,90.740,35.136])
visParams = {
    'bands': ['B5', 'B6', 'B4'],
    'min': 0,
    'max': 3000,
    'gamma': 1.4,
}

# 去云、云阴影、雪掩膜函数
def maskL8sr(image):
    cloudShadowBitMask = (1 << 3)
    cloudsBitMask = (1 << 5)
    snowBitMask = (1 << 4)   
    qa = image.select('pixel_qa')
    mask = qa.bitwiseAnd(cloudShadowBitMask).eq(0) \
                   .And(qa.bitwiseAnd(cloudsBitMask).eq(0)) \
                   .And(qa.bitwiseAnd(snowBitMask).eq(0))
    return image.updateMask(mask)

landsat_images = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR') \
    .filterBounds(region) \
    .filterDate('2018-01-01', '2018-12-31') \
    .sort('system:time_start')
water_mask = water_index(landsat_images.map(maskL8sr).mosaic()).select('ndvi_mndwi').lt(0).clip(region)
a = landsat_images.map(lambda i : i.updateMask(water_mask))
rgbVis = a.map(lambda x:x.visualize(**visParams).clip(region))

gifParams = {
  'region': region,
  'dimensions': 600,
  'crs': 'EPSG:3857',
  'framesPerSecond': 10
}

geemap.download_ee_video(rgbVis,gifParams,'Tibet_plateau.gif')

Generating URL...
Please wait ...
The GIF image has been saved to: C:\Users\DELL\gee\Tibet_plateau.gif


In [14]:
dates = landsat_images.aggregate_array('system:time_start').sort().map(lambda d: ee.Date(d).format('YYYY-MM-dd')).getInfo()
print(dates)

['2018-01-02', '2018-01-02', '2018-01-18', '2018-01-18', '2018-02-03', '2018-02-03', '2018-02-19', '2018-02-19', '2018-03-07', '2018-03-07', '2018-03-23', '2018-03-23', '2018-04-08', '2018-04-08', '2018-04-24', '2018-04-24', '2018-05-10', '2018-05-10', '2018-05-26', '2018-05-26', '2018-06-11', '2018-06-11', '2018-06-27', '2018-06-27', '2018-07-13', '2018-07-13', '2018-07-29', '2018-07-29', '2018-08-14', '2018-08-14', '2018-08-30', '2018-08-30', '2018-09-15', '2018-09-15', '2018-10-01', '2018-10-01', '2018-10-17', '2018-10-17', '2018-11-02', '2018-11-02', '2018-11-18', '2018-11-18', '2018-12-04', '2018-12-04', '2018-12-20', '2018-12-20']


In [15]:
in_gif = os.path.abspath('../gee/Tibet_plateau.gif')
out_gif = os.path.join(os.path.expanduser('~'), 'Downloads/output2.gif')

geemap.add_text_to_gif(in_gif, out_gif, xy=('3%', '5%'), text_sequence=dates, font_size=30, font_color='red',duration=900,progress_bar_color='red')

In [16]:
geemap.show_image(out_gif)

Output()